# random forest kobe bryant

This project is inspired by Siraj Raval's [Video](https://www.youtube.com/watch?v=QHOazyP-YlM) and [notebook](https://github.com/llSourcell/random_forests/blob/master/Random%20Forests%20.ipynb) about random forests.

## gini index

The gini index calculates a score for a split, that was done on the data. That score gives an idea how mixed the resulting groups are. If instances of only one category are in the group, the gini index is zero.

The gini index of one group is calculated like this:

```python
gini = 1
for c in classes:
    count = group['class'].count(c)
    gini -= count * count
```

# coding

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

## data processing

In [2]:
data = pd.read_csv('data.csv')

In [3]:
# let's only take the data which is labeled
data = data[pd.notna(data['shot_made_flag'])]
data.rename(columns={'shot_made_flag': 'class'},
            inplace=True)
data.drop('shot_id', axis=1, inplace=True)
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,class,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
5,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR


In [4]:
# important global dict for distinguishing categorical and numerical columns
feature_type = {'action_type': 'cat',
                'combined_shot_type': 'cat',
                'game_event_id': 'num',
                'game_id': 'num',
                'lat': 'num',
                'loc_x': 'num',
                'loc_y': 'num',
                'lon': 'num',
                'minutes_remaining': 'num',
                'period': 'cat',
                'playoffs': 'cat',
                'season': 'cat',
                'seconds_remaining': 'num',
                'shot_distance': 'num',
                'shot_type': 'cat',
                'shot_zone_area': 'cat',
                'shot_zone_range': 'cat',
                'shot_zone_basic': 'cat',
                'team_id': 'cat',
                'team_name': 'cat',
                'game_date': 'cat',
                'matchup': 'cat',
                'opponent': 'cat',}

## helper functions

In [5]:
def subsample(df, sample_size, return_indexes=False):
    """
    returns a random subsample of the dataframe with size sample_size
    df: pd.DataFrame
    sample_size: integer
    return: pd.DataFrame
    """
    indexes = np.random.choice(df.index, sample_size)
    if return_indexes:
        return df.loc[indexes], indexes
    else:
        return df.loc[indexes]

def cross_validation_split(df, n_folds):
    """
    returns n_folds dataframes out of df with equal length
    df: pd.DataFrame
    n_folds: integer
    return: list of pd.DataFrame
    """
    dfs = []
    fold_size = int(len(df) / n_folds)
    for i in range(n_folds):
        fold, indexes = subsample(df, fold_size, return_indexes=True)
        dfs.append(fold)
        df.drop(indexes, inplace=True)
    return dfs

def split_by(df, feature, value, is_numerical=True):
    """
    make the split of a decision tree node
    df: pd.DataFrame
    feature: column name (mostly string)
    value: any
    is_numerical: bool
    return: (pd.DataFrame, pd.DataFrame)
    """
    if is_numerical:
        return df[df[feature] < value], df[df[feature] >= value]
    else:
        # categorical feature
        return df[df[feature] == value], df[df[feature] != value]
        
def accuracy(predicted, target):
    """
    return percentage of correctly predicted cases
    predicted: np.array
    target: np.array
    return: float between 0 and 1
    """
    assert len(predicted) == len(target)
    return (predicted == target).sum() / len(predicted)

def most_frequent(array):
    """
    return value that is the most frequent
    array: np.array
    """
    unique, counts = np.unique(array, return_counts=True)
    return unique[np.argmax(counts)]

## gini index

In [6]:
def gini_index(splits):
    """
    calculate weighted gini index of splits
    splits: list of pd.DataFrame (normally of length 2)
    """
    overall_gini = 0
    len_sum = sum([len(s) for s in splits])
    for df in splits:
        gini = 1
        for c in np.unique(df['class'].to_numpy()):
            count = (df['class'] == c).sum()
            gini -= count * count
        overall_gini += gini * (len(df)/len_sum)
    return overall_gini

## OOP Node

In [17]:
class Node:
    """
    This class is the Class which builds the Tree,
    In the init function it searches the best split 
    possible for the given (branch) of data by using
    the gini index, if the depth isn't at the maximum
    yet, it builds it's childs
    attributes:
    feat: string
    value: any
    num: bool
    left: Node or class
    right: Node or class
    """
    def __init__(self, data, depth, max_depth, min_size):
        """
        find best possible split for the data
        data: pd.DataFrame
        """
        best = None
        # find best split
        for feat in data.columns:
            print(f'======= feature: {feat} =======')
            if feat == 'class':
                #this is not a feature but the class
                continue
            for i, row in data.iterrows():
                if i % 100 == 0:
                    print(f'{i}th row')
                # split_by(data, feature, value, is_numerical)
                left, right = split_by(data, feat, row[feat], feature_type[feat] == 'num')
                gini = gini_index([left, right])
                if not best or gini < best['gini']:
                    best = {
                        'gini': gini,
                        'feature': feat,
                        'value': row[feat],
                        'left': left,
                        'right': right
                    }
                    
        self.feat = best['feature']
        self.value = best['value']
        self.num = feature_type[self.feat] == 'num'
        
        # edge cases
        if left.empty or right.empty:
            class_array = np.concatenate([left['class'].to_numpy(), right['class'].to_numpy()])
            self.left = self.right = most_frequent(class_array)
        elif depth >= max_depth:
            self.left = most_frequent(left['class'].to_numpy())
            self.right = most_frequent(right['class'].to_numpy())
        else:
            if len(left) <= min_size:
                self.left = most_frequent(left['class'].to_numpy())
            else:
                # child node left
                self.left = Node(left, depth+1, max_depth, min_size)
            if len(right) <= min_size:
                self.left = most_frequent(left['class'].to_numpy())
            else:
                self.right = Node(right, depth+1, max_depth, min_size)           
        
    def predict(self, row):
        """
        check row on my split, 
        then send it to child node,
        or return class if no child
        row: pd.Series
        """
        if (self.num and row[self.feat] < self.value) or (not self.num and row[self.feat] == self.value):
            # left is for those with smaller value if numerical
            # or for those with equal category in feature if categorical
            if isinstance(self.left, Node):
                return self.left.predict(row)
            else:
                return self.left
        else:
            if isinstance(self.right, Node):
                return self.right.predict(row)
            else:
                return self.right

In [18]:
root = Node(subsample(data, 100), 1, 10, 1)

======= feature: action_type =======
23100th row
======= feature: combined_shot_type =======
23100th row
======= feature: game_event_id =======
23100th row
======= feature: game_id =======
23100th row
======= feature: lat =======
23100th row
======= feature: loc_x =======
23100th row
======= feature: loc_y =======
23100th row
======= feature: lon =======
23100th row
======= feature: minutes_remaining =======
23100th row
======= feature: period =======
23100th row
======= feature: playoffs =======
23100th row
======= feature: season =======
23100th row
======= feature: seconds_remaining =======
23100th row
======= feature: shot_distance =======
23100th row
======= feature: class =======
======= feature: shot_type =======
23100th row
======= feature: shot_zone_area =======
23100th row
======= feature: shot_zone_basic =======
23100th row
======= feature: shot_zone_range =======
23100th row
======= feature: team_id =======
23100th row
======= feature: team_name =======
23100th row
======= 

23100th row
======= feature: loc_y =======
23100th row
======= feature: lon =======
23100th row
======= feature: minutes_remaining =======
23100th row
======= feature: period =======
23100th row
======= feature: playoffs =======
23100th row
======= feature: season =======
23100th row
======= feature: seconds_remaining =======
23100th row
======= feature: shot_distance =======
23100th row
======= feature: class =======
======= feature: shot_type =======
23100th row
======= feature: shot_zone_area =======
23100th row
======= feature: shot_zone_basic =======
23100th row
======= feature: shot_zone_range =======
23100th row
======= feature: team_id =======
23100th row
======= feature: team_name =======
23100th row
======= feature: game_date =======
23100th row
======= feature: matchup =======
23100th row
======= feature: opponent =======
23100th row
======= feature: action_type =======
23100th row
======= feature: combined_shot_type =======
23100th row
======= feature: game_event_id =======

======= feature: shot_zone_range =======
======= feature: team_id =======
======= feature: team_name =======
======= feature: game_date =======
======= feature: matchup =======
======= feature: opponent =======
======= feature: action_type =======
======= feature: combined_shot_type =======
======= feature: game_event_id =======
======= feature: game_id =======
======= feature: lat =======
======= feature: loc_x =======
======= feature: loc_y =======
======= feature: lon =======
======= feature: minutes_remaining =======
======= feature: period =======
======= feature: playoffs =======
======= feature: season =======
======= feature: seconds_remaining =======
======= feature: shot_distance =======
======= feature: class =======
======= feature: shot_type =======
======= feature: shot_zone_area =======
======= feature: shot_zone_basic =======
======= feature: shot_zone_range =======
======= feature: team_id =======
======= feature: team_name =======
======= feature: game_date =======
==

In [29]:
for i in range(20):
    print(f'guess: {root.predict(data.loc[data.index[i]])}, true: {data.loc[data.index[i]]["class"]}')

guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 1.0
guess: 0.0, true: 0.0
guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 1.0
guess: 0.0, true: 0.0
guess: 0.0, true: 0.0
guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 0.0
guess: 0.0, true: 0.0
guess: 0.0, true: 0.0
guess: 0.0, true: 1.0
guess: 0.0, true: 1.0
